# Machine Vision mit Yolov5

Yolov5 ist eine Objekterkennungs Architektur in der fünften Version die von der Firma Ultralytics kostenlos bereitgestellt wird. 

Die Software kann unter der GNU General Public License v3.0 unter gewissen vorraussetzungen auch Kommerziel genutzt werden.


### Erste Schritte
#### Setup in conda
1. Eine Conda umgebung in der Python Version 3.7.11 wird benötigt
2. in der conda Umgebung pip, jupyter, cudatoolkit und lxml installieren
3. -> conda install pytorch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2 cudatoolkit=10.1 -c pytorch
4. in den aktuellen yolov5 ordner wechseln mit "cd .\yolov5\"
5. benötigte bibliotheken mit "pip install -r requirements.txt" installieren




Das Repository kann über diesen [Link](https://github.com/ultralytics/yolov5) heruntergeladen werden. Alternativ kann es auch über Git geklont werden.
Hier sind schon alle Dateien die benötigt werden also ist der vorhgerige Schritt hier nicht nötig.

In [ ]:
# Import 
import cv2 
import os
import glob
import shutil

In [ ]:
## Startet die Kamera zum Bilder erstellen

  
# Videoaufnahme
def useCameraForPictureCapture(train = True):
    # Festlegen der Bildernamen
    iTrain = len(os.listdir('./yolov5/data/images/train/images'))
    iValidation = len(os.listdir('./yolov5/data/images/validation/images'))
    # Auslesen der Kamera
    vid = cv2.VideoCapture(0) 
    
    while(True): 
        ret, frame = vid.read() 

        cv2.imshow('frame', frame) 
        if cv2.waitKey(1) & 0xFF == ord('s'):
            if train:
                cv2.imwrite('./yolov5/data/images/train/images/' + f'{iTrain:05d}'+ '.jpeg', frame)
                iTrain+=1   
            else:
                cv2.imwrite('./yolov5/data/images/validation/images/' + f'{iValidation:05d}'+ '.jpeg', frame)
                iValidation+=1
            
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
        
    
    vid.release() 
    cv2.destroyAllWindows() 

### Trainingsbilder aufnehmen (training set)

Hierbei ist es überproportional wichtig die Klassen (Objekte) die erkannt werden sollen, möglichst von allen Seiten und mit unterschiedlichem Hintergründen zu fotografieren.

In [ ]:
useCameraForPictureCapture()
# Fenster schließen = q Taste
# Bild aufnehmen = s Taste

### Validierungsbilder aufnehmen (validation set) 
Ein gutes Verhältnis zwischen Trainings- und Validierungsbildern ist Beispielsweise 5:1 (5 Trainingsbilder pro Validierungsbild)

In [ ]:
useCameraForPictureCapture(False)
# Fenster schließen = q Taste
# Bild aufnehmen = s Taste

###

### Festlegen der Klassen

Um den Bildern Klassen zuordnen zu können, müssen zunächst die sogenannten Klassen festgelegt werden.
Dazu müssen zwei Dateien bearbeitet werden.

1. Einfügen der Anzahl der Klassen und Klassennamen in der "/yolov5/data/coco128.yaml" Datei
2. In der Datei "/OpenLabeling/main/class_list.txt" müssen die Klassennamen auch aufgeführt werden. Hier pro Klassenname eine Zeile (Keine Kommata)


### Klassifizierung (labeling) der Bilder
Wichtig beim klassifizieren der Bilder ist es die Kästen (Bounding Boxes) möglichst genau um das Objekt zu zeichnen, da Ungenauigkeiten beim klassifizieren auch zu Ungenauigkeiten bei der Objekterkennung führen.

Steuerung des Tools:

<img src="OpenLabeling\keyboard_usage.jpg" alt="keyboardUsage" title="Keyboard Usage" width="150" height="100" />

Die beiden commandos unten legen die Eingabepfade für die Bilder und die Ausgabepfade für die Labels fest. (Können prinzipiell genauso in der Konsole ausgeführt werden solange das Machine-Vision verzeichniss geöffnet ist)

In [ ]:
!python OpenLabeling/main/main.py --input_dir ../../yolov5/data/images/train/images --output_dir ../../yolov5/data/images/train/labels

### Klassifizierung der Validierungsdaten

In [ ]:
!python OpenLabeling/main/main.py --input_dir ../../yolov5/data/images/validation/images --output_dir ../../yolov5/data/images/validation/labels

### Trainieren des Models

Als letzter Schritt muss dass Model in Yolo Trainiert werden.

Dabei können unterschiedliche Parametern (sogenannten Flags) spezifiziert werden. 

Hier sind einige Beispiele für Flags in Yolov5:
1. "--data ..." spezifiziert die yaml Datei die zum trainieren genutzt wird
2. "--batch-size ..." spezifiziert die Anzahl der Datenpunkte die durch das Netzwerk geführt werden
3. "--epochs ..." spezifiziert die Anzahl der Trainierten Epochen
4. "--weights ..." spezifiziert das vortrainierte yolo dataset welches genutzt wird um das

Bevor das Model trainiert werden kann, müssen die Label noch in den Korrekten Ordner verschoben werden. Dies wird von den beiden unteren schleifen erledigt.

In [ ]:
for filename in glob.glob(os.path.join("yolov5/data/images/train/labels/YOLO_darknet/", '*.*')):
    shutil.copy(filename, "yolov5/data/images/train/labels/")

for filename in glob.glob(os.path.join("yolov5/data/images/validation/labels/YOLO_darknet/", '*.*')):
    shutil.copy(filename, "yolov5/data/images/validation/labels/")

In [ ]:
!python yolov5/train.py --batch 20 --epochs 10 --data coco128.yaml --weights yolov5s.pt